In [38]:
import pandas as pd

In [39]:
book_shop_df = pd.read_csv("https://raw.githubusercontent.com/danielpm1982/sample-mysql-python-pandas-data-engineering/refs/heads/master/books.csv")
book_shop_df.head()

# empty values, at the .csv columns, will be read as 'NaN', by Pandas read_csv() funcion. This "NaN" type will match all empty 
# values when merging two dataframes - as done in the sequence. Either you change the NaN values to an equivalent string (such as 
# "Unknown") before creating the second df from this first one, so that the two dfs would keep matching entries accordingly, or 
# you would let to change - the NaN to a string - only after the merge (join) is done, in order to not hinder the matching of 
# these values between the two dfs. Here, we'll choose this second approach: we'll rename the NaN values only after the merge, 
# but before exporting these values to the populate sql files.
# print(f"{len(book_shop_df[book_shop_df['Genre'].isna()])} rows with empty Genre values, read as NaN:")
# book_shop_df[book_shop_df["Genre"].isna()]

,Book,Author(s),Original language,First published,Approximate sales in millions,Genre
0,A Tale of Two Cities,Charles Dickens,English,1859,200.0,Historical fiction
1,The Little Prince (Le Petit Prince),Antoine de Saint-Exupéry,French,1943,200.0,Novella
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120.0,Fantasy
3,And Then There Were None,Agatha Christie,English,1939,100.0,Mystery
4,Dream of the Red Chamber (紅樓夢),Cao Xueqin,Chinese,1791,100.0,Family saga


In [40]:
book_shop_df.shape

(174, 6)

In [41]:
book_shop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Book                           174 non-null    object 
 1   Author(s)                      174 non-null    object 
 2   Original language              174 non-null    object 
 3   First published                174 non-null    int64  
 4   Approximate sales in millions  174 non-null    float64
 5   Genre                          118 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 8.3+ KB


In [42]:
book_shop_df.columns

Index(['Book', 'Author(s)', 'Original language', 'First published',
       'Approximate sales in millions', 'Genre'],
      dtype='object')

In [43]:
column_mapper_dict = {
    "Book": "BOOK",
    "Author(s)": "AUTHOR",
    "Original language": "LANGUAGE",
    "First published": "FIRST_PUBLISHED",
    "Approximate sales in millions": "SALES (MILLIONS OF US$)",
    "Genre": "GENRE"
}
book_shop_df.rename(columns=column_mapper_dict, inplace=True)
book_shop_df.head()

,BOOK,AUTHOR,LANGUAGE,FIRST_PUBLISHED,SALES (MILLIONS OF US$),GENRE
0,A Tale of Two Cities,Charles Dickens,English,1859,200.0,Historical fiction
1,The Little Prince (Le Petit Prince),Antoine de Saint-Exupéry,French,1943,200.0,Novella
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120.0,Fantasy
3,And Then There Were None,Agatha Christie,English,1939,100.0,Mystery
4,Dream of the Red Chamber (紅樓夢),Cao Xueqin,Chinese,1791,100.0,Family saga


In [44]:
book_shop_df.isnull().sum()

# the NaN values at the Genre column will be treated only after the merge between the unique_genre_df with the book_shop_df, 
# later below. Before exporting the populate sql statements to the sql file (so that the DB registries will have the proper 
# string "Unknown" instead of 'nan').
# No treatment is needed for the Author(s) column's values though, as there are no empty (NaN) values for it at the csv or df.

BOOK                        0
AUTHOR                      0
LANGUAGE                    0
FIRST_PUBLISHED             0
SALES (MILLIONS OF US$)     0
GENRE                      56
dtype: int64

In [45]:
book_shop_df["AUTHOR"].value_counts().head(10)

AUTHOR
J. K. Rowling       7
Dan Brown           3
Suzanne Collins     3
Roald Dahl          3
George Orwell       2
Ken Follett         2
E. L. James         2
Albert Camus        2
Erskine Caldwell    2
Cao Xueqin          1
Name: count, dtype: int64

In [46]:
book_shop_df[book_shop_df["AUTHOR"]=="J. K. Rowling"]

,BOOK,AUTHOR,LANGUAGE,FIRST_PUBLISHED,SALES (MILLIONS OF US$),GENRE
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120.0,Fantasy
10,Harry Potter and the Chamber of Secrets,J. K. Rowling,English,1998,77.0,Fantasy
11,Harry Potter and the Prisoner of Azkaban,J. K. Rowling,English,1999,65.0,Fantasy
12,Harry Potter and the Goblet of Fire,J. K. Rowling,English,2000,65.0,Fantasy
13,Harry Potter and the Order of the Phoenix,J. K. Rowling,English,2003,65.0,Fantasy
14,Harry Potter and the Half-Blood Prince,J. K. Rowling,English,2005,65.0,Fantasy
15,Harry Potter and the Deathly Hallows,J. K. Rowling,English,2007,65.0,Fantasy


In [47]:
unique_author_df = pd.DataFrame(book_shop_df["AUTHOR"].unique(), columns=["AUTHOR"])
unique_author_df["AUTHOR_ID"] = unique_author_df.index + 1
unique_author_df.head(10)

,AUTHOR,AUTHOR_ID
0,Charles Dickens,1
1,Antoine de Saint-Exupéry,2
2,J. K. Rowling,3
3,Agatha Christie,4
4,Cao Xueqin,5
5,J. R. R. Tolkien,6
6,C. S. Lewis,7
7,H. Rider Haggard,8
8,Ved Prakash Sharma,9
9,Dan Brown,10


In [48]:
book_shop_df = book_shop_df.merge(unique_author_df, on="AUTHOR", how="left")
book_shop_df.head(20)

,BOOK,AUTHOR,LANGUAGE,FIRST_PUBLISHED,SALES (MILLIONS OF US$),GENRE,AUTHOR_ID
0,A Tale of Two Cities,Charles Dickens,English,1859,200.0,Historical fiction,1
1,The Little Prince (Le Petit Prince),Antoine de Saint-Exupéry,French,1943,200.0,Novella,2
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120.0,Fantasy,3
3,And Then There Were None,Agatha Christie,English,1939,100.0,Mystery,4
4,Dream of the Red Chamber (紅樓夢),Cao Xueqin,Chinese,1791,100.0,Family saga,5
5,The Hobbit,J. R. R. Tolkien,English,1937,100.0,Fantasy,6
6,"The Lion, the Witch and the Wardrobe",C. S. Lewis,English,1950,85.0,"Fantasy, Children's fiction",7
7,She: A History of Adventure,H. Rider Haggard,English,1887,83.0,Adventure,8
8,Vardi Wala Gunda (वर्दी वाला गुंडा),Ved Prakash Sharma,Hindi,1992,80.0,Detective,9
9,The Da Vinci Code,Dan Brown,English,2003,80.0,Mystery thriller,10


In [49]:
book_shop_df[book_shop_df["AUTHOR"]=="J. K. Rowling"]

,BOOK,AUTHOR,LANGUAGE,FIRST_PUBLISHED,SALES (MILLIONS OF US$),GENRE,AUTHOR_ID
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120.0,Fantasy,3
10,Harry Potter and the Chamber of Secrets,J. K. Rowling,English,1998,77.0,Fantasy,3
11,Harry Potter and the Prisoner of Azkaban,J. K. Rowling,English,1999,65.0,Fantasy,3
12,Harry Potter and the Goblet of Fire,J. K. Rowling,English,2000,65.0,Fantasy,3
13,Harry Potter and the Order of the Phoenix,J. K. Rowling,English,2003,65.0,Fantasy,3
14,Harry Potter and the Half-Blood Prince,J. K. Rowling,English,2005,65.0,Fantasy,3
15,Harry Potter and the Deathly Hallows,J. K. Rowling,English,2007,65.0,Fantasy,3


In [50]:
unique_genre_df = pd.DataFrame(book_shop_df["GENRE"].unique(), columns=["GENRE"])
unique_genre_df["GENRE_ID"] = unique_genre_df.index + 1
unique_genre_df

,GENRE,GENRE_ID
0,Historical fiction,1
1,Novella,2
2,Fantasy,3
3,Mystery,4
4,Family saga,5
...,...,...
76,"Historical fiction, war novel",77
77,Erotica,78
78,Science fiction,79
79,Dystopian fiction,80


In [51]:
book_shop_df = book_shop_df.merge(unique_genre_df, on="GENRE", how="left")
book_shop_df.head(10)

,BOOK,AUTHOR,LANGUAGE,FIRST_PUBLISHED,SALES (MILLIONS OF US$),GENRE,AUTHOR_ID,GENRE_ID
0,A Tale of Two Cities,Charles Dickens,English,1859,200.0,Historical fiction,1,1
1,The Little Prince (Le Petit Prince),Antoine de Saint-Exupéry,French,1943,200.0,Novella,2,2
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120.0,Fantasy,3,3
3,And Then There Were None,Agatha Christie,English,1939,100.0,Mystery,4,4
4,Dream of the Red Chamber (紅樓夢),Cao Xueqin,Chinese,1791,100.0,Family saga,5,5
5,The Hobbit,J. R. R. Tolkien,English,1937,100.0,Fantasy,6,3
6,"The Lion, the Witch and the Wardrobe",C. S. Lewis,English,1950,85.0,"Fantasy, Children's fiction",7,6
7,She: A History of Adventure,H. Rider Haggard,English,1887,83.0,Adventure,8,7
8,Vardi Wala Gunda (वर्दी वाला गुंडा),Ved Prakash Sharma,Hindi,1992,80.0,Detective,9,8
9,The Da Vinci Code,Dan Brown,English,2003,80.0,Mystery thriller,10,9


In [52]:
book_shop_df[book_shop_df["AUTHOR"]=="J. K. Rowling"]

,BOOK,AUTHOR,LANGUAGE,FIRST_PUBLISHED,SALES (MILLIONS OF US$),GENRE,AUTHOR_ID,GENRE_ID
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120.0,Fantasy,3,3
10,Harry Potter and the Chamber of Secrets,J. K. Rowling,English,1998,77.0,Fantasy,3,3
11,Harry Potter and the Prisoner of Azkaban,J. K. Rowling,English,1999,65.0,Fantasy,3,3
12,Harry Potter and the Goblet of Fire,J. K. Rowling,English,2000,65.0,Fantasy,3,3
13,Harry Potter and the Order of the Phoenix,J. K. Rowling,English,2003,65.0,Fantasy,3,3
14,Harry Potter and the Half-Blood Prince,J. K. Rowling,English,2005,65.0,Fantasy,3,3
15,Harry Potter and the Deathly Hallows,J. K. Rowling,English,2007,65.0,Fantasy,3,3


In [53]:
unique_author_df = unique_author_df.drop("AUTHOR_ID", axis=1)
unique_author_df

# as with the GENRE_ID from the unique_genre_df, we won't need the AUTHOR_ID from the unique_author_df anymore... therefore, we 
# can simply remove this whole column from the unique_author_df. These ID columns, at the unique_author_df and unique_genre_df 
# were meant to simulate what the DBMS will do by autoincrement, regarding the respective PK values... and to leave a 
# correspondent column at the book_shop_df, as FK (through the merge), in order to stablish the associations at the DB.

,AUTHOR
0,Charles Dickens
1,Antoine de Saint-Exupéry
2,J. K. Rowling
3,Agatha Christie
4,Cao Xueqin
...,...
152,Eliyahu M. Goldratt
153,Ray Bradbury
154,Frank McCourt
155,Mohandas Karamchand Gandhi


In [54]:
with open("author_table_populate.sql", "w", encoding="utf-8") as f:
    for _, row in unique_author_df.iterrows():
        name = row["AUTHOR"].replace("'", "''")
        f.write(f"INSERT INTO AUTHOR(NAME) VALUES ('{name}');\n")

In [55]:
book_shop_df.isnull().sum()

BOOK                        0
AUTHOR                      0
LANGUAGE                    0
FIRST_PUBLISHED             0
SALES (MILLIONS OF US$)     0
GENRE                      56
AUTHOR_ID                   0
GENRE_ID                    0
dtype: int64

In [56]:
book_shop_df["GENRE"] = book_shop_df["GENRE"].fillna("Unknown")

# we could have left the NaN values at the GENRE column of the book_shop_df, as this column won't be exported to the sql file or 
# DB. What we gotta treat are the NaN values at the GENRE column at the unique_genre_df, which are the values which are going to 
# be exported to the DB.

In [57]:
book_shop_df.isnull().sum()

BOOK                       0
AUTHOR                     0
LANGUAGE                   0
FIRST_PUBLISHED            0
SALES (MILLIONS OF US$)    0
GENRE                      0
AUTHOR_ID                  0
GENRE_ID                   0
dtype: int64

In [58]:
unique_genre_df["GENRE"] = unique_genre_df["GENRE"].fillna("Unknown")

# now, all registries pointing out to this GENRE value (GENRE_ID=58), at the DB, will point out to "Unknown", instead of to a NaN 
# or nan values... which is much more semantic.

In [59]:
unique_genre_df.isnull().sum()

GENRE       0
GENRE_ID    0
dtype: int64

In [60]:
unique_genre_df[unique_genre_df["GENRE"].isna()]

,GENRE,GENRE_ID


In [61]:
unique_genre_df[unique_genre_df["GENRE"]=='Unknown']

,GENRE,GENRE_ID
57,Unknown,58


In [62]:
unique_genre_df = unique_genre_df.drop("GENRE_ID",axis=1)
unique_genre_df

# as with the AUTHOR_ID from the unique_author_df, we won't need the GENRE_ID from the unique_genre_df anymore, either... thus, 
# we can simply remove this whole column from the unique_genre_df, as well. Again, these ID columns, at the unique_author_df and
# unique_genre_df were meant to simulate what the DBMS will do by autoincrement, regarding the respective PK values... and to 
# leave a correspondent column at the book_shop_df, as FK (through the merge), in order to stablish the associations at the DB.

,GENRE
0,Historical fiction
1,Novella
2,Fantasy
3,Mystery
4,Family saga
...,...
76,"Historical fiction, war novel"
77,Erotica
78,Science fiction
79,Dystopian fiction


In [63]:
unique_genre_df["GENRE"] = unique_genre_df["GENRE"].str.lower();
unique_genre_df

# setting all GENRE names to lowercase, to avoid letter-case variations

,GENRE
0,historical fiction
1,novella
2,fantasy
3,mystery
4,family saga
...,...
76,"historical fiction, war novel"
77,erotica
78,science fiction
79,dystopian fiction


In [64]:
with open("genre_table_populate.sql", "w", encoding="utf-8") as f:
    for _, row in unique_genre_df.iterrows():
        name = str(row["GENRE"]).replace("'", "''")
        f.write(f"INSERT INTO GENRE(NAME) VALUES ('{name}');\n")

In [65]:
with open("book_table_populate.sql", "w", encoding="utf-8") as f:
    for _, row in book_shop_df.iterrows():
        book = row["BOOK"].replace("'", "''")
        language = row["LANGUAGE"].replace("'", "''")
        first_published = int(row["FIRST_PUBLISHED"])
        sales = float(row["SALES (MILLIONS OF US$)"])
        author_id = int(row["AUTHOR_ID"])
        genre_id = int(row["GENRE_ID"])
        sql = f"INSERT INTO BOOK(NAME, LANGUAGE, FIRST_PUBLISHED, SALES, AUTHOR_ID, GENRE_ID) VALUES ('{book}', '{language}',{first_published},{sales:.2f},{author_id},{genre_id});\n"
        f.write(sql)

In [66]:
feedback_data = 'https://raw.githubusercontent.com/guilhermeonrails/datas-csv/refs/heads/main/comentarios.json'
feedback_df = pd.read_json(feedback_data)
feedback_df.head(10)

,id,livro,nome,sobrenome,comentario
0,1,The Dukan Diet,Danielle,Johnson,Achei a história envolvente e bem escrita.
1,2,"The Lion, the Witch and the Wardrobe",Joshua,Walker,Esperava mais do enredo.
2,3,James and the Giant Peach,Jill,Rhodes,Personagens cativantes e trama surpreendente.
3,4,The Very Hungry Caterpillar,Patricia,Miller,Achei a história envolvente e bem escrita.
4,5,Bridget Jones's Diary,Robert,Johnson,Um dos melhores que já li.
5,6,Lolita,Jeffery,Wagner,"Boa leitura, mas não é meu estilo."
6,7,Matilda,Anthony,Gonzalez,"Uma leitura incrível, recomendo!"
7,8,She: A History of Adventure,Debra,Gardner,Achei a história envolvente e bem escrita.
8,9,Gone with the Wind,Jeffrey,Lawrence,Personagens cativantes e trama surpreendente.
9,10,A Wrinkle in Time,Lisa,Smith,"Boa leitura, mas não é meu estilo."


In [67]:
feedback_df.shape

(10000, 5)

In [68]:
book_shop_df.head()

,BOOK,AUTHOR,LANGUAGE,FIRST_PUBLISHED,SALES (MILLIONS OF US$),GENRE,AUTHOR_ID,GENRE_ID
0,A Tale of Two Cities,Charles Dickens,English,1859,200.0,Historical fiction,1,1
1,The Little Prince (Le Petit Prince),Antoine de Saint-Exupéry,French,1943,200.0,Novella,2,2
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120.0,Fantasy,3,3
3,And Then There Were None,Agatha Christie,English,1939,100.0,Mystery,4,4
4,Dream of the Red Chamber (紅樓夢),Cao Xueqin,Chinese,1791,100.0,Family saga,5,5


In [69]:
if "LIVRO_ID" not in feedback_df.columns:
    feedback_df = feedback_df.merge(
        book_shop_df[["BOOK"]].reset_index(drop=False).rename(columns={"index": "LIVRO_ID"}),
        left_on="livro",
        right_on="BOOK",
        how="left"
    )
    feedback_df["LIVRO_ID"] +=1
else:
    print("* This merge has already been done before and the feedback_df already contains LIVRO_ID column. New merge ignored !\n")
feedback_df.head()

,id,livro,nome,sobrenome,comentario,LIVRO_ID,BOOK
0,1,The Dukan Diet,Danielle,Johnson,Achei a história envolvente e bem escrita.,164,The Dukan Diet
1,2,"The Lion, the Witch and the Wardrobe",Joshua,Walker,Esperava mais do enredo.,7,"The Lion, the Witch and the Wardrobe"
2,3,James and the Giant Peach,Jill,Rhodes,Personagens cativantes e trama surpreendente.,63,James and the Giant Peach
3,4,The Very Hungry Caterpillar,Patricia,Miller,Achei a história envolvente e bem escrita.,36,The Very Hungry Caterpillar
4,5,Bridget Jones's Diary,Robert,Johnson,Um dos melhores que já li.,174,Bridget Jones's Diary


In [70]:
# Now that the association between the feedback_df and book_shop_df has been defined - through the merge - we no longer need 
# either "livro", "BOOK" or the "id" columns at the feedback_df, only the LIVRO_ID column (other than nome, sobrenome and 
# comentario). LIVRO_ID will be the FK needed for mapping the feedbacks to the correct DB instances at the BOOK table. All PKs,
# including the PK for the FEEDBACK_PT_BR table, will be auto-created by the SGBD (they're not exported from here). The FKs, 
# however, have been created and set here already, between the associated dfs, and will be exported at the sql statements to the 
# actual DB. The simulation used here regarding the PK auto-generation (that only happens at the SGBD), in order to set the 
# corresponding FKs at the associated dfs, necessarily assumes that the auto-generation will follow the same pattern (sequenced 
# int numbers starting from 1). If the auto-generation at the SGBD follows another pattern, all FKs here will 
# point out to the wrong auto-generated PKs. This risk must be clear and well considered by the data engineer and DBAs who will 
# manage the DB squema. Anyone who manages the schema of the DB must know that all PKs must start at 1 and be an int sequence
# from the first until the last registry there, for all tables.

feedback_df = feedback_df.drop("livro", axis=1)
feedback_df = feedback_df.drop("BOOK", axis=1)
feedback_df = feedback_df.drop("id", axis=1)

In [71]:
feedback_df.isnull().sum()

nome          0
sobrenome     0
comentario    0
LIVRO_ID      0
dtype: int64

In [72]:
def format_value(value):
    if pd.isna(value):
        return "NULL"
    elif isinstance(value, str):
        value = value.replace("'","''")
        return f"'{value}'"
    else:
        return str(value)

In [73]:
with open("feedback_pt_br_table_populate.sql", "w", encoding="utf-8") as f:
    for _, row in feedback_df.iterrows():
        values = (
            format_value(row["LIVRO_ID"]),
            format_value(row["nome"]),
            format_value(row["sobrenome"]),
            format_value(row["comentario"])
        )
        sql = f"INSERT INTO FEEDBACK_PT_BR(LIVRO_ID, NOME_CLIENTE, SOBRENOME_CLIENTE, FEEDBACK) VALUES ({', '.join(values)});\n"
        f.write(sql)

## Nested left join of BOOK, AUTHOR and GENRE tables:

### selecting all fields (except FK for avoiding redundance on columns):
select b.ID as 'BOOK_ID', b.NAME as 'BOOK_NAME', b.LANGUAGE as 'BOOK_ORIGINAL_LANGUAGE', b.FIRST_PUBLISHED, b.SALES as 'SALES_MILLIONS_OF_US$', a.ID as AUTHOR_ID, a.NAME as 'AUTHOR_NAME', g.ID as 'GENRE_ID', g.NAME as 'GENRE_NAME' from BOOK b left join AUTHOR a on b.AUTHOR_ID=a.ID left join GENRE g on b.GENRE_ID=g.ID;

### creating a View for this selection query:
CREATE VIEW BOOK_AUTHOR_GENRE_JOIN_VIEW AS
select b.ID as 'BOOK_ID', b.NAME as 'BOOK_NAME', b.LANGUAGE as 'BOOK_ORIGINAL_LANGUAGE', b.FIRST_PUBLISHED, b.SALES as 'SALES_MILLIONS_OF_US$', a.ID as AUTHOR_ID, a.NAME as 'AUTHOR_NAME', g.ID as 'GENRE_ID', g.NAME as 'GENRE_NAME' from BOOK b left join AUTHOR a on b.AUTHOR_ID=a.ID left join GENRE g on b.GENRE_ID=g.ID;

### checking out all Views at the database:
SHOW FULL TABLES WHERE table_type = 'VIEW';

### checking out the view content above:
select * from BOOK_AUTHOR_GENRE_JOIN_VIEW;

### selecting only business fields (no PK fields):
select b.NAME as 'BOOK_NAME', b.LANGUAGE as 'BOOK_ORIGINAL_LANGUAGE', b.FIRST_PUBLISHED, b.SALES as 'SALES_MILLIONS_OF_US$', a.NAME as 'AUTHOR_NAME', g.NAME as 'GENRE_NAME' from BOOK b left join AUTHOR a on b.AUTHOR_ID=a.ID left join GENRE g on b.GENRE_ID=g.ID;

### printscreen of view 1 above:
![Select View 1](https://raw.githubusercontent.com/danielpm1982/sample-mysql-python-pandas-data-engineering/refs/heads/master/select_view_1.png)


## Left join of FEEDBACK_PT_BR and BOOK_AUTHOR_GENRE_JOIN_VIEW tables:

### selecting all fields (except FK for avoiding redundance on columns):
select f.ID as 'FEEDBACK_ID', f.NOME_CLIENTE, f.SOBRENOME_CLIENTE, f.FEEDBACK, v.BOOK_ID, v.BOOK_NAME, v.BOOK_ORIGINAL_LANGUAGE, v.FIRST_PUBLISHED, v.SALES_MILLIONS_OF_US$, v.AUTHOR_ID, v.AUTHOR_NAME, v.GENRE_ID, v.GENRE_NAME from FEEDBACK_PT_BR f left join BOOK_AUTHOR_GENRE_JOIN_VIEW v on f.LIVRO_ID=v.BOOK_ID;

### creating a View for this selection query:
CREATE VIEW FEEDBACK_BOOK_AUTHOR_GENRE_JOIN_VIEW AS
select f.ID as 'FEEDBACK_ID', f.NOME_CLIENTE, f.SOBRENOME_CLIENTE, f.FEEDBACK, v.BOOK_ID, v.BOOK_NAME, v.BOOK_ORIGINAL_LANGUAGE, v.FIRST_PUBLISHED, v.SALES_MILLIONS_OF_US$, v.AUTHOR_ID, v.AUTHOR_NAME, v.GENRE_ID, v.GENRE_NAME from FEEDBACK_PT_BR f left join BOOK_AUTHOR_GENRE_JOIN_VIEW v on f.LIVRO_ID=v.BOOK_ID;

### checking out all Views at the database:
SHOW FULL TABLES WHERE table_type = 'VIEW';

### checking out the view content above:
select * from FEEDBACK_BOOK_AUTHOR_GENRE_JOIN_VIEW;

### selecting only business fields (no PK fields):
select f.NOME_CLIENTE, f.SOBRENOME_CLIENTE, f.FEEDBACK, v.BOOK_NAME, v.BOOK_ORIGINAL_LANGUAGE, v.FIRST_PUBLISHED, v.SALES_MILLIONS_OF_US$, v.AUTHOR_NAME, v.GENRE_NAME from FEEDBACK_PT_BR f left join BOOK_AUTHOR_GENRE_JOIN_VIEW v on f.LIVRO_ID=v.BOOK_ID;

### printscreen of view 2 above:
![Select View 2](https://raw.githubusercontent.com/danielpm1982/sample-mysql-python-pandas-data-engineering/refs/heads/master/select_view_2.png)

.

# TO BE CONTINUED AND FINALLY DOCUMENTED ...


.